# EXPORTED THE MONGODB COLLECTION
I exported the `iwp` collection in the `etl_db` to the file `mongodb_export_iwp.json` using
```mongoexport --db etl_db --collection iwp --out mongodb_export_iwp.json```


# IMPORT THE MONGODB COLLECTION
The JSON can be imported to the `iwp` collection in the `etl_db` database from file `mongodb_export_iwp.json` using
```mongoimport --db etl_db --collection iwp --file mongodb_export_iwp.json```


# DOCUMENT FORMAT
The documents in the etl_db collection iwp have the following formats:
1. United States - Address is parsed
<br>

```{'_id': ObjectId('5c6a8db6aaa45be6f85938c7'),
 'incident_title': "BJ's Restaurant & Brewhouse, Red Bug Lake Road, Oviedo, FL, USA",
 'incident_date': 'Dec 30 2018 10:51pm',
 'incident_url': 'https://iwaspoisoned.com/incident/bjs-restaurant-brewhouse-red-bug-lake-road-oviedo-fl-usa-155865#emailscroll',
 'incident_report_type': 'Food Poisoning',
 'incident_symptoms': ['Diarrhea', 'Nausea', 'Vomiting'],
 'incident_misc': '',
 'incident_address_info':
     {'incident_address': '8285 Red Bug Lake Road, Oviedo, 32765 Florida, United States',
      'incident_address_standard': '8285 Red Bug Lake Road, Oviedo, Florida 32765, United States',
      'incident_address_street': '8285 Red Bug Lake Road',
      'incident_address_street2': '',
      'incident_address_street3': '',
      'incident_address_city': 'Oviedo',
      'incident_address_state': 'Florida',
      'incident_address_zipcode': '32765',
      'incident_address_country': 'United States'},
 'incident_description': 'Ate there Friday evening, got sick about 8 hours later'}```

<br>


2. Other Countries - Address is not parsed
(Should only USA entries in the database due to filtering, but just in case...)
<br>

```{'_id': ObjectId('5c6a8db6aaa45be6f85938c7'),
 'incident_title': "BJ's Restaurant & Brewhouse, Red Bug Lake Road, Oviedo, FL, USA",
 'incident_date': 'Dec 30 2018 10:51pm',
 'incident_url': 'https://iwaspoisoned.com/incident/bjs-restaurant-brewhouse-red-bug-lake-road-oviedo-fl-usa-155865#emailscroll',
 'incident_report_type': 'Food Poisoning',
 'incident_symptoms': ['Diarrhea', 'Nausea', 'Vomiting'],
 'incident_misc': '',
 'incident_address_info':
     {'incident_address': '8285 Red Bug Lake Road, Oviedo, 32765 Florida, United States'},
 'incident_description': 'Ate there Friday evening, got sick about 8 hours later'}```

<br>


In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
from pprint import pprint

from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo

import time

# Support export to and import from JSON file
import json

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.etl_db

# Convert the items in MongoDB collection
# to a flat format permit import to DataFrame
iwp_info = db.iwp.find()

new_i_list = []

for i in iwp_info:
    # Create a new list of dicts with flat entries
    #print(i['incident_title'])
    new_dict = {
         'incident_title': i['incident_title'],
         'incident_date': i['incident_date'],
         'incident_url': i['incident_url'],
         'incident_report_type': i['incident_report_type'],
         'incident_symptoms': ", ".join(i['incident_symptoms']),
         'incident_misc': i['incident_misc'],
          'incident_address': i['incident_address_info']['incident_address'],
          'incident_address_standard': i['incident_address_info']['incident_address_standard'],
          'incident_address_street': i['incident_address_info']['incident_address_street'],
          'incident_address_street2': i['incident_address_info']['incident_address_street2'],
          'incident_address_street3': i['incident_address_info']['incident_address_street3'],
          'incident_address_city': i['incident_address_info']['incident_address_city'],
          'incident_address_state': i['incident_address_info']['incident_address_state'],
          'incident_address_zipcode': i['incident_address_info']['incident_address_zipcode'],
          'incident_address_country': i['incident_address_info']['incident_address_country'],
         'incident_description': i['incident_description']
         }
    
    new_i_list.append(new_dict)

In [3]:
# Try to import the dictionary into pandas
iwp_df = pd.DataFrame(new_i_list)
iwp_df.head()

,incident_address,incident_address_city,incident_address_country,incident_address_standard,incident_address_state,incident_address_street,incident_address_street2,incident_address_street3,incident_address_zipcode,incident_date,incident_description,incident_misc,incident_report_type,incident_symptoms,incident_title,incident_url
0,"6152 95th Street, Oak Lawn, 60453 Illinois, Un...",Oak Lawn,United States,"6152 95th Street, Oak Lawn, Illinois 60453, Un...",Illinois,6152 95th Street,,,60453,Feb 18 2019 1:44am,Had a quick dinner 5:00 Thursday February 14th...,,Food Poisoning,"Diarrhea, Fever, Nausea, Vomiting","Panda Express, 95th Street, Oak Lawn, IL, USA",https://iwaspoisoned.com/incident/panda-expres...
1,"6380 Grand Avenue, Gurnee, 60031 Illinois, Uni...",Gurnee,United States,"6380 Grand Avenue, Gurnee, Illinois 60031, Uni...",Illinois,6380 Grand Avenue,,,60031,Feb 16 2019 10:15pm,"Picked up a Jalepeno McChicken, Bacon McDouble...",,Food Poisoning,"Diarrhea, Vomiting, Nausea","McDonald's, 6380 Grand Ave, Gurnee, IL 60031, USA",https://iwaspoisoned.com/incident/mcdonalds-63...
2,"2700 Crego Road, DeKalb, 60115 Illinois, Unite...",DeKalb,United States,"2700 Crego Road, DeKalb, Illinois 60115, Unite...",Illinois,2700 Crego Road,,,60115,Feb 14 2019 10:35pm,Ordered 3 chicken strips. Took forever.....4 h...,,Food Poisoning,"Diarrhea, Nausea","Popeyes Louisiana Kitchen, Crego Road, DeKalb,...",https://iwaspoisoned.com/incident/popeyes-loui...
3,"725 U.S. 24, Gilman, 60938 Illinois, United St...",Gilman,United States,"725 U.S. 24, Gilman, Illinois 60938, United St...",Illinois,725 U.S. 24,,,60938,Feb 13 2019 7:33pm,Ate there around 11pm on 2/12/19 and by 500 am...,,Food Poisoning,"Diarrhea, Fever, Nausea, Vomiting","Burger King, U.S. 24, Gilman, IL, USA",https://iwaspoisoned.com/incident/burger-king-...
4,"1301 Court Street, Pekin, 61554 Illinois, Unit...",Pekin,United States,"1301 Court Street, Pekin, Illinois 61554, Unit...",Illinois,1301 Court Street,,,61554,Feb 16 2019 12:57am,I ordered a beef and cheddar sandwich and my b...,,Food Poisoning,Diarrhea,"Arby's, Court Street, Pekin, IL, USA",https://iwaspoisoned.com/incident/arbys-court-...
